In [5]:
'''
Created on 2010-04-17
@author: rv
'''

import pygame
from pygame.locals import *
import random
from itertools import product
from environment import Environment
from agent import Agent
from wdgPopulation import WdgPopulation
from wdgWealth import WdgWealth
from wdgAgent import WdgAgent

''' 
initial simulation parameters
'''

# view
screenSize = 600, 600
colorBackground = 250,250,250

colorSugar = ((250,250,220),
              (250,250,200),
              (250,250,180),
              (250,250,160),
              (250,250,140),
              (250,250,120),
              (250,250,100),
              (250,250,80),
              (250,250,60),
              (250,250,40))

colorRed = 250, 50, 50
colorPink = 250, 50, 250
colorBlue = 50, 50, 250

# environment
gridSize = 50, 50
northSite = 40, 10, 20
southSite = 15, 35, 20
maxCapacity = 10 # !!! < or = nbr items in colorSugar array
seasonPeriod = 50
northRegion = 0, 0, 49, 24
southRegion = 0, 25, 49, 49
growFactor = 1
growFactor1 = 1
growFactor2 = float(growFactor1) / 8

# agents
# agentColorScheme: Agents colour meaning = 0:all, 1:bySexe, 2:byMetabolism, 3:byVision, 4:byGroup
maxAgentMetabolism = 4
maxAgentVision = 6
initEndowment = 50, 100
minmaxAgentAge = 60, 100
female = 0
male = 1
fertility = [(12, 15, 40, 50),
             (12, 15, 50, 60),
             (12, 15, 30, 40), 
             (12, 15, 40, 50)]
childbearing = fertility[0], fertility[1]   # female , male
tagsLength = 5 # must be odd
tags0 = 0
tags1 = 2**tagsLength - 1

''' settings for Evolution from random distribution
agentColorScheme = 0        
distributions = [(400, None, (0, 50, 0, 50))] 
ruleGrow = True
ruleSeasons = False
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = False'''

''' settings for Emergent waves migration
agentColorScheme = 0        
distributions = [(300, None, (0, 20, 30, 50))]
ruleGrow = True
ruleSeasons = False
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = False'''

''' settings for Seasonal migration
agentColorScheme = 0        
distributions = [(400, None, (0, 50, 0, 50))] 
ruleGrow = True
ruleSeasons = True
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = False'''

''' settings for societal evolution
agentColorScheme = 3       
distributions = [(300, None, (0, 50, 0, 50))] 
ruleGrow = True
ruleSeasons = False
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = True
ruleReplacement = False
ruleProcreate = True
ruleTransmit = False'''

''' settings for Cultural transmission
agentColorScheme = 4
distributions = [(200, tags0, (0, 50, 0, 50)), (200, tags1, (0, 50, 0, 50))]
ruleGrow = True
ruleSeasons = False
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = True'''

'''# settings for Combat with alpha = infinite
agentColorScheme = 4
distributions = [
    (300, tags0, (0, 20, 30, 50)),  # blues
    (300, tags1, (30, 50, 0, 20))]  # reds
ruleGrow = True
ruleSeasons = False
ruleMoveEat = False
ruleCombat = True
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = False
combatAlpha = 1000000'''

'''# settings for Trench war with alpha = 2
agentColorScheme = 4
distributions = [
    (300, tags0, (0, 20, 30, 50)),  # blues
    (300, tags1, (30, 50, 0, 20))]  # reds
ruleGrow = True
ruleSeasons = False
ruleMoveEat = False
ruleCombat = True
ruleLimitedLife = False
ruleReplacement = True
ruleProcreate = False
ruleTransmit = False
combatAlpha = 2'''

'''# settings for Combat eliminates Waves with alpha = 2
agentColorScheme = 4
distributions = [
    (300, tags0, (0, 20, 30, 50)),  # blues
    (300, tags1, (30, 50, 0, 20))]  # reds
ruleGrow = True
ruleSeasons = False
ruleMoveEat = False
ruleCombat = True
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = False
combatAlpha = 2'''

'''# settings for Combat and cultural transmission
agentColorScheme = 4
distributions = [
    (300, tags0, (0, 20, 30, 50)),  # blues
    (300, tags1, (30, 50, 0, 20))]  # reds
ruleGrow = True
ruleSeasons = False
ruleMoveEat = False
ruleCombat = True
ruleLimitedLife = False
ruleReplacement = False
ruleProcreate = False
ruleTransmit = True
combatAlpha = 1000000'''

# settings for Proto-History
agentColorScheme = 4
distributions = [
    (25, tags0, (0, 50, 0, 50)),  # blues
    (25, tags1, (0, 50, 0, 50))]  # reds
ruleGrow = True
ruleSeasons = False
ruleMoveEat = True
ruleCombat = False
ruleLimitedLife = True
ruleReplacement = False
ruleProcreate = True
ruleTransmit = True
combatAlpha = 1000000

fps = 10

''' 
Global functions
'''

def initAgent(agent, tags, distribution):
    newLocation = agent.getEnv().getRandomFreeLocation(distribution)
    if newLocation == None:
        return False
    agent.setLocation(newLocation)
    agent.setMetabolism(random.randint(1, maxAgentMetabolism))
    agent.setVision(random.randint(1, maxAgentVision))
    agent.setInitialEndowment(random.randint(initEndowment[0], initEndowment[1]))
    agent.setAge(random.randint(minmaxAgentAge[0], minmaxAgentAge[1]))
    sexe = random.randint(0,1)
    agent.setSexe(sexe)
    agent.setFertility((random.randint(childbearing[sexe][0],childbearing[sexe][1]), random.randint(childbearing[sexe][2],childbearing[sexe][3])))
    if tags == None:
        tags = random.getrandbits(tagsLength)
    agent.setTags((tags, tagsLength))
    return True
    
''' 
View Class
'''
class View: 
    
    # this gets called first
    def __init__(self, screenSize, env, agents):
        # init view
        pygame.init()
        pygame.display.set_caption("Sugarscape")
        self.screenSize = screenSize
        self.screen = pygame.display.set_mode(screenSize)
        self.quit = False
        self.siteSize = screenSize[0] / env.gridWidth
        self.radius = int(self.siteSize * 0.5)
        # init env
        self.env = env
        self.season = ""
        # init agents population
        self.agents = agents
        self.population = [len(self.agents)]
        self.metabolismMean = []
        self.visionMean = []
        # init time
        self.iteration = 0
        self.clock = pygame.time.Clock()

    # display agent switch case (dictionary)
    def all(self, agent):
        return colorRed

    def bySexe(self, agent):
        if agent.getSexe() == female:
            return colorPink
        else:
            return colorBlue
    
    def byMetabolism(self, agent):
        if agent.getMetabolism() > 2:
            return colorRed
        else:
            return colorBlue
    
    def byVision(self, agent):
        if agent.getVision() > 3:
            return colorRed
        else:
            return colorBlue

    def byGroup(self, agent):
#        if bin(agent.getTags()).count('1') > agent.getTagsLength()>>1:
        if agent.getTribe() == 1:
            return colorRed
        else:
            return colorBlue

    agentColorSchemes = {0:all, 1:bySexe, 2:byMetabolism, 3:byVision, 4:byGroup}
    
    # remove or replace agent
    def findDistribution(self, tags):
        getTribe = lambda x, y: round(float(bin(x).count('1')) / float(y))
        tribe = getTribe(tags, tagsLength)
        for (n, t, d) in distributions:
            if t != None and getTribe(t, tagsLength) == tribe:
                # found a distribution for tags
                return d
        else:
            # or return best guess
            return d

    # replace or remove agent
    def removeAgent(self, agent):
        if ruleReplacement:
            # replace with agent of same tribe
            tags = agent.getTags()
            if initAgent(agent, tags, self.findDistribution(tags)):
                self.env.setAgent(agent.getLocation(), agent)
            else:
                print("initAgent failed!")
                self.agents.remove(agent)
        else:
            self.agents.remove(agent)
    
    # put game update code here
    def update(self):
        # for agents' logs
        metabolism = 0 
        vision = 0 
        
        # execute agents randomly
        random.shuffle(self.agents)
        
        # run agents' rules
        for agent in self.agents:
            # MOVE
            if ruleMoveEat:
                agent.move()
                # remove agent if he's dead
                if agent.getSugar() == 0:
                    # free environment
                    self.env.setAgent(agent.getLocation(), None)
                    # remove or replace agent
                    self.removeAgent(agent)
                    continue
                
            # COMBAT
            if ruleCombat:
                killed = agent.combat(combatAlpha)
                # if an agent has been killed, remove it
                if killed:
                    # do not free the environment, someone else is already here
                    self.removeAgent(killed)
                # remove agent if he's dead
                if agent.getSugar() == 0:
                    # free environment
                    self.env.setAgent(agent.getLocation(), None)
                    # remove or replace agent
                    self.removeAgent(agent)
                    continue
            
            # PROCREATE
            if ruleProcreate and agent.isFertile():
                mateItr = agent.mate()
                while True:
                    try:
                        # if a new baby is born, append it to the agents' list
                        self.agents.append(mateItr.next())
                    except StopIteration:
                        break
            
            # TRANSMIT
            if ruleTransmit:
                agent.transmit()
            
            # Log agent's parameters
            metabolism += agent.getMetabolism()
            vision += agent.getVision()

            # DIE
            # increment age
            alive = agent.incAge();
            if ruleLimitedLife and not alive:
                # free environment
                self.env.setAgent(agent.getLocation(), None)
                # remove or replace agent
                self.removeAgent(agent)

        # Log population
        numAgents = len(self.agents)
        self.population.append(numAgents)
        
        # Calculate and log agents' metabolism and vision mean values
        if numAgents > 0:
            self.metabolismMean.append(metabolism/float(numAgents))
            self.visionMean.append(vision/float(numAgents))

        # run environment's rules
        if ruleSeasons:
            S = (self.iteration % (2 * seasonPeriod)) / seasonPeriod
            if S < 1:
                # Summer
                self.season = "(summer, winter)"
                if ruleGrow:
                    self.env.growRegion(northRegion,growFactor1)
                    self.env.growRegion(southRegion,growFactor2)
            else:
                # winter
                self.season = "(winter, summer)"
                if ruleGrow:
                    self.env.growRegion(northRegion,growFactor2)
                    self.env.growRegion(southRegion,growFactor1)
        elif ruleGrow:
            self.season = "NA"
            self.env.grow(growFactor)

    # put drawing code here
    def draw(self):
        self.screen.fill(colorBackground)

        # display sugarscape
        for i, j in product(range(env.gridHeight), range(env.gridWidth)):
            x = i * self.siteSize
            y = j * self.siteSize
            # display sugar's capacity
            capacity = env.getCapacity((i,j))
            if capacity > 0:
                pygame.draw.rect(self.screen, colorSugar[capacity - 1], (x, y, self.siteSize - 1, self.siteSize - 1))

            # Draw agent if any
            agent = env.getAgent((i, j))
            if agent:
                pygame.draw.circle(self.screen,
                                    # select color scheme
                                    self.agentColorSchemes.get(agentColorScheme)(self, agent), 
                                    (x + self.radius, y + self.radius), 
                                    self.radius - 1)

        pygame.display.flip()

    # the main game loop
    def mainLoop(self):
        dt = 0
        framecount = 0
        framerate = 0
        update = False
        while not self.quit:
            t0 = pygame.time.get_ticks()
            # handle events
            for event in pygame.event.get():
                if event.type == QUIT:
                    self.quit = True
                    
                elif event.type == KEYDOWN:
                    
                    if event.key == K_F1:
                        # Display agents' population widget
                        widget = WdgPopulation(self.population, "Population time series", 1000, 300)
                        widget.execute()
                        
                    elif event.key == K_F2:
                        # Display agents' wealth widget 
                        widget = WdgWealth(self.agents, "Wealth histogram", 500, 500)
                        widget.execute()

                    elif event.key == K_F3:
                        # Display agents' wealth widget 
                        widget = WdgAgent(self.metabolismMean, self.visionMean, "Agents' metabolism and vision mean values", 1000, 300)
                        widget.execute()

                    elif event.key == K_F12:
                        update = not update

            # update sugarscape
            if update:
                self.update()
                self.iteration += 1
            
            # display sugarscape state
            self.draw()

            # wait simulation step
            self.clock.tick(fps)

            # calculate and display the framerate
            t1 = pygame.time.get_ticks()
            dt += t1 - t0
            framecount += 1
            if dt >= 1000:
                dt -= 1000
                framerate = framecount
                framecount = 0
                
            # display infos
            if update:
                print( "Iteration = ", self.iteration, "; fps = ", framerate, "; Seasons (N,S) = ", self.season, "; Population = ", len(self.agents), " -  press F12 to pause.")

''' 
Main 
'''

if __name__ == '__main__' :
    
    env = Environment(gridSize)
    
    # add radial food site 
    env.addFoodSite(northSite, maxCapacity)
    
    # add radial food site 
    env.addFoodSite(southSite, maxCapacity)

    # grow to max capacity
    if ruleGrow:
        env.grow(maxCapacity)

    # create a lit of agents and place them in env
    agents = []
    for (numAgents, tags, distribution) in distributions:
        for i in range(numAgents):
            agent = Agent(env)
            if initAgent(agent, tags, distribution):
                env.setAgent(agent.getLocation(), agent)
                agents.append(agent)
    
    # Create a view with an env and a list of agents in env
    view = View(screenSize, env, agents)
    
    # iterate
    view.mainLoop()

ModuleNotFoundError: No module named 'pygame'

In [ ]:
import random
from itertools import product

class Agent:
    '''
    classdocs
    '''
    costForMatingRange = 0,0 #10,40
    tagsProbability = 0.55
    
    def __init__(self, env, x = 0, y = 0, metabolism = 4, vision = 6, endowment = 25, maxAge = 100, sexe = 0, fertility = (12,50), tags = (0, 11)):
        '''
        Constructor
        '''
        self.env = env
        self.x = x
        self.y = y
        self.metabolism = metabolism
        self.vision = vision
        self.maxAge = maxAge
        self.age = 0
        self.sex = sex
        self.fertility = fertility
        self.setInitialEndowment(endowment)
        self.setTags(tags)

    ''' 
    get / set section 
    '''
    def getEnv(self):
        return self.env
    
    def setLocation(self,x, y):
        self.x = x
        self.y = y

    def getLocation(self):
        return (self.x, self.y)
    
    def setMetabolism(self, metabolism):
        self.metabolism = metabolism
        
    def getMetabolism(self):
        return self.metabolism

    def setVision(self, vision):
        self.vision = vision
        
    def getVision(self):
        return self.vision

    def setInitialEndowment(self, endowment):
        self.endowment = endowment
        self.sugar = endowment
        if self.costForMatingRange == (0,0):
            self.costForMating = endowment
        else:
            self.costForMating = random.randint(self.costForMatingRange[0], self.costForMatingRange[1])
        
    def getSugar(self):
        return self.sugar

    def setAge(self, maxAge):
        self.maxAge = maxAge
        self.age = 0
        
    def getAge(self):
        return self.age
    
    def setSexe(self, sexe):
        self.sexe = sexe
        
    def getSexe(self):
        return self.sexe
    
    def setFertility(self, fertility):
        self.fertility = fertility
        
    def setTags(self, tags, tagsLength):
        self.tags = tags
        self.tagsLength = tagsLength
        self.tribe = round(float(bin(tags).count('1')) / float(tagsLength))
    
    def getTags(self):
        return self.tags

    def getTagsLength(self):
        return self.tagsLength
    
    def getTribe(self):
        return self.tribe

    ''' 
    build common lists
    '''

    # build a list of available food locations
    def getFood(self):
        food = [(x, self.y) for x in range(self.x - self.vision, self.x + self.vision + 2)
                if self.env.isLocationValid((x, self.y)) 
                and self.env.isLocationFree((x, self.y))]

        food.extend([(self.x, y) for y in range (self.y - self.vision, self.y + self.vision + 2) 
                if self.env.isLocationValid((self.x, y)) 
                and self.env.isLocationFree((self.x, y))])
        return food
    
    # build a list of possible neighbours for in neighbourhood
    def getNeighbourhood(self):
        neighbourhood = [self.env.getAgent((x,self.y)) for x in range(self.x - 1, self.x + 2)
                         if self.env.isLocationValid((x, self.y)) 
                         and not self.env.isLocationFree((x, self.y))
                         and x != self.x]
        
        neighbourhood.extend([self.env.getAgent((self.x,y)) for y in range(self.y - 1, self.y + 2)
                         if self.env.isLocationValid((self.x, y)) 
                         and not self.env.isLocationFree((self.x, y))
                         and y != self.y])
        return neighbourhood

    # build a list of possible preys around
    def getPreys(self):
        preys = [self.env.getAgent((x, self.y)) for x in range(self.x - self.vision, self.x + self.vision + 2)
                if self.env.isLocationValid((x, self.y)) 
                and not self.env.isLocationFree((x, self.y))
                and self.sugar > self.env.getAgent((x, self.y)).getSugar()
                and self.env.getAgent((x, self.y)).getTribe() != self.tribe]

        preys.extend([self.env.getAgent((self.x, y)) for y in range(self.y - self.vision, self.y + self.vision + 2)
                if self.env.isLocationValid((self.x, y)) 
                and not self.env.isLocationFree((self.x, y))
                and self.sugar > self.env.getAgent((self.x, y)).getSugar()
                and self.env.getAgent((self.x, y)).getTribe() != self.tribe])
        return preys
    
    ''' 
    rules
    '''

    # TRANSMIT
    def transmit(self):
        # build a list of possible neighbours for in neighbourhood
        neighbourhood = self.getNeighbourhood()

        # tag-flipping with neighbours
        for neighbour in neighbourhood:
            mask = 1 << random.randint(0, self.tagsLength - 1)
            neighbourTags = neighbour.getTags()
            if  (self.tags & mask) != (neighbourTags & mask):
                # flip neighbour's tag
                neighbourTags ^= mask
                # transmit new tag
                neighbour.setTags((neighbourTags, self.tagsLength))

    # AGEING
    # If age > maxAge Then the agent is dead (return False)
    def incAge(self):
        self.age += 1
        return (max(self.maxAge - self.age, 0))
    
    # FERTILITY
    # First, to have offspring, agents must be of childbearing age. 
    # Second, children born with literally no initial endowment of sugar would instantly die. 
    # We therefore require that parents give their children some initial endowment.
    # Each newborn's endowment is the sum of the (usually unequal) contributions of mother and father.
    # Dad contributes an amount equal to one half of whatever his initial endowment had been, and likewise for mom.
    # To be parents, agents must have amassed at least the amount of sugar which they were endowed at birth.
    def isFertile(self):
        return (self.age >= self.fertility[0] and self.age <= self.fertility[1] and self.sugar >= self.costForMating)

    # MATE (Generator):
    # Select a neighboring agent at random.
    # If the neighbor is fertile and of the opposite sex and at least one of the agent has an empty neighboring site, then a child is born
    # Repeat for all neighbors.
    def mate(self):
        # build a list of possible partners in neighbourhood
        neighbourhood = self.getNeighbourhood()
        
        # randomize
        random.shuffle(neighbourhood)
        
        # mate with (all) possible partners
        for neighbour in neighbourhood:
            # partner selection
            if neighbour.getSexe() == self.sexe or not neighbour.isFertile():
                continue
            # find a free location around the agent for the baby 
            freeLocation = self.findFreeLocationAround(self.x, self.y)
            if not freeLocation:
                # or find around the partner
                freeLocation = self.findFreeLocationAround(neighbour.x, neighbour.y)
            # then, give birth if a location has been found
            if freeLocation:
                yield self.createChild(neighbour, freeLocation)

    # Find a free location around x,y (for baby)
    def findFreeLocationAround(self, x, y):
        '''locations = [(i, j) for i,j in product(range(x - 1, x + 2), range(y - 1, y + 2))
                     if self.env.isLocationValid((i, j)) 
                     and self.env.isLocationFree((i, j))]'''
        locations = [(i, y) for i in range(x - 1, x + 2) if self.env.isLocationValid((i, y)) and self.env.isLocationFree((i, y))]
        locations.extend([(x, j) for j in range(y - 1, y + 2) if self.env.isLocationValid((x, j)) and self.env.isLocationFree((x, j))])
        length = len(locations)
        if length:
            return locations[random.randint(0, length - 1)]
        return None
    
    # Cross-over offspring from two parents: parent1 and parent2
    def createChild(self, parent, x, y):
        # cross-over parents genetics
        genitors = [self, parent]
        metabolism = genitors[random.randint(0,1)].metabolism
        vision = genitors[random.randint(0,1)].vision
        endowment = 0.5 * (genitors[0].endowment + genitors[1].endowment)
        genitors[0].sugar = max(genitors[0].sugar - 0.5 * genitors[0].endowment, 0)
        genitors[1].sugar = max(genitors[1].sugar - 0.5 * genitors[1].endowment, 0)
        ageMax = genitors[random.randint(0,1)].maxAge
        sexe = random.randint(0,1)
        fertility = genitors[random.randint(0,1)].fertility
        
        # build cultural tags from parents genetics
        mask = 1
        childTags = 0
        for tag in range(self.tagsLength):
            tag1 = self.tags & mask
            tag2 = parent.getTags() & mask
            if tag1 == tag2 or random.random() < self.tagsProbability:
                childTags |= tag1
            else:
                childTags |= tag2
            mask <<= 1
 
        # create child agent
        child = Agent(self.env, x, y, metabolism, vision, endowment, ageMax, sexe, fertility, (childTags, self.tagsLength))
        self.env.setAgent((x, y), child)
        return child

                
    # MOVE:
    # Look out as far as vision permits in the four principal lattice directions and identify the unoccupied site(s) having the most sugar.
    # If the greatest sugar value appears on multiple sites then select the nearest one.
    # Move to this site.
    # Collect all the sugar at this new position.
    # Increment the agent's accumulated sugar wealth by the sugar collected and decrement by the agent's metabolic rate.

    def move(self):
        # build a list of available food locations
        food = self.getFood()

        # randomize food locations
        random.shuffle(food)

        # find best food location
        # much faster than sorting.
        move = False
        newx = self.x
        newy = self.y
        best = self.env.getCapacity((self.x, self.y))
        minDistance = 0
        for (x,y) in food:
            capacity = self.env.getCapacity((x, y))
            distance = abs(x - self.x + y - self.y) # Manhattan distance enough due to no diagonal
            if capacity > best or (capacity == best and distance < minDistance):
                best = capacity
                minDistance = distance
                move = True
                newx = x
                newy = y

        # move to new location if any
        if move:
            self.env.setAgent((self.x, self.y), None)
            self.env.setAgent((newx, newy), self)
            self.x = newx
            self.y = newy
            
        # collect, eat and consume
        self.sugar = max(self.sugar + best - self.metabolism, 0)
        self.env.setCapacity((self.x, self.y), 0)

    '''
    # sort implementation
    # (for the record: actually was slower so not used)
    def compareLocations(self, (x1, y1), (x2, y2)):
        c1 = self.env.getCapacity((x1, y1))
        c2 = self.env.getCapacity((x2, y2))
        if c1 - c2:
            # bigger is better
            return cmp(c1, c2)
        else:
            # Manhattan distance enough due to no diagonal
            d1 = abs(x1 - self.x + y1 - self.y)
            d2 = abs(x2 - self.x + y2 - self.y)
            # shorter is better
            return cmp(d2, d1)
    
    def moveSort(self):
        # build a list of available food locations
        food = getFood()
        
        # randomize food locations
        random.shuffle(food)
        if len(food):
            # add existing location (self.x, self.y) => currently not in food
            food.extend([(self.x, self.y)])
            # randomize food locations
            random.shuffle(food)
            # sort by capacity and distance
            food.sort(cmp = self.compareLocations, reverse = True)
            # move to best location
            (newx, newy) = food[0]
            self.env.setAgent((self.x, self.y), None)
            self.env.setAgent((newx, newy), self)
            self.x = newx
            self.y = newy
            # collect, eat and consume
            self.sugar = max(self.sugar + self.env.getCapacity(self.x, self.y) - self.metabolism, 0)
            self.env.setCapacity((self.x, self.y), 0)'''
        
    # COMBAT
    def combat(self, alpha):
        # build a list of available unoccupied food locations
        food = self.getFood()
        
        # build a list of potential preys
        preys = self.getPreys()
        
        # append to food safe preys (retaliation condition)
        C0 = self.sugar - self.metabolism
        food.extend([preyA.getLocation() for preyA, preyB in product(preys, preys)
                     if preyA != preyB 
                     and preyB.getSugar() < (C0 + self.env.getCapacity(preyA.getLocation()) + min(alpha, preyA.getSugar()))])
        
        # randomize food locations
        random.shuffle(food)

        # find best food location
        move = False
        newx = self.x
        newy = self.y
        best = self.env.getCapacity((self.x, self.y))
        minDistance = 0
        for (x,y) in food:
            capacity = self.env.getCapacity((x, y))
            agent = self.env.getAgent((x, y))
            if agent :
                capacity += min(alpha, agent.getSugar())
            distance = abs(x - self.x + y - self.y) # Manhattan distance enough due to no diagonal
            if capacity > best or (capacity == best and distance < minDistance):
                best = capacity
                minDistance = distance
                move = True
                newx = x
                newy = y

        # move to new location if any, and kill if occupied
        killed = None
        if move:
            killed = self.env.getAgent((newx, newy))
            self.env.setAgent((self.x, self.y), None)
            self.env.setAgent((newx, newy), self)
            self.x = newx
            self.y = newy
            
        # collect, eat and consume
        self.sugar = max(self.sugar + best - self.metabolism, 0)
        self.env.setCapacity((self.x, self.y), 0)
        return killed

In [1]:
?isLocationValid

Object `isLocationValid` not found.


In [36]:
class Habitante(Agente):
    
    def __init__(self, x, y, metabolismo = random.randint(1,8), wealth):
        Agente.__init__(self, tipo)
        self.wealth = wealth
        self.metabolismo = metabolismo
        self.sugar = sugar
        self.locacion = np.array([random.uniform(0, 1), random.uniform(0, 1)]) # Posición al azar inicial




    def vecinos(self, agentes):
        pass

    def actuar(self, agentes):
        self.locacion = np.array([random.uniform(0, 1), random.uniform(0, 1)]) 


In [ ]:
class Sugarscape(Mundo):
    def __init__(self):
        self.agentes = agentes
        self.ancho = ancho
        self.alto = alto
        self.steps = steps
        self.locacion = np.array([(x, y) for x in range(ancho) for y in range(alto)]) # Posición al azar inicial azúcar

    def vacio(self):
        return not ()
    
    def ocupado(self):
        return ()
    
    def random_cell(self):
        return randint(0, self.ancho - 1), randint(0, self.alto - 1)
    
    def distancia(self, otro):
        "Calcula la distancia euclídea entre este agente y otro"
        return np.linalg.norm(self.locacion - otro.locacion)
    
    def random_empty_cell(self):
        candidato = self.random_cell()
        while self.occupied(*candidato):
            candidato = self.random_cell()
        return candidate

    def lugar(self, item, x, y):
        coords = self.coords(x, y)
        self._by_coords[coords] = item
        self._by_item[item] = coords
    
    
        

        





In [ ]:
class SugarLocation():
    def __init__(self, sugar, max_sugar, x, y):
        self.max_sugar = max_sugar
        self.sugar = sugar
        self.posicion = (x, y)
        self.vecinos = []
        self.setSugar(0.5, 40)
    
    def setSugar(self):
        if self.sugar < self.max_sugar:
            self.sugar = random.randint(1,6)
        else: 
            self.sugar = 0
            

In [52]:
class Bien(Sugarscape, Habitante):
    def __init__(self, alto, ancho, growth_rate, tipo):
        Sugarscape.__init__(self, alto, ancho)
        Habitantes.__init__(self, tipo)
        self.growth_rate = growth_rate
        self.max_sugar = max_sugar
        self.sugar = 0

    def grow(self, cantidad):
        if self.sugar < self.max_sugar:
            self.sugar += cantidad
            self.sugar = min(self.sugar, self.max_sugar)
            
            
    
    